## Cálculo del Scoring

El scoring $S_{ijk}$ es un valor que multiplica a cada variable $X_{ijk}$ con el fin de aumentar o disminuir la probabilidad de ser incluida en el modelo.

>$S_{ijk}$: scoring aplicado al tipo de vehículo $i$ en la estación $j$ y en el turno $k$. Cada valor de $S_{ijk}$ se calcula de la siguiente manera:

$S_{ijk} = P_{BarriosCercanos_j}*S_{BarriosCercanos_j} + P_{Densidad_j}*S_{Densidad_j} + P_{Capacidad_j}*S_{Capacidad_j} + P_{Estaciones Cerca_j}*S_{Estaciones Cerca_j} + P_{Incidencias Turno_k}*S_{Incidencias Turno_k} + P_{Vehículo_{i,k}}*S_{Vehículo_{i,k}}$.

$S_{ijk} \geq{0};\ S_{ijk} \in \mathbb{R} ;\ S_{ijk} \in [0,1]$

---

Donde $P_{x}$ es el peso asociado a cada scoring $S_{x}$. Los valores de $P_{x}$ se sitúan en el intervalo de `[0,1]`. Dichos pesos serán los siguientes:

- $P_{BarriosCercanos_j} = 0.25$
- $P_{Densidad_j} = 0.05$
- $P_{Capacidad_j} = 0.15$
- $P_{Estaciones Cerca_j} = 0.2$
- $P_{Incidencias Turno_k} = 0.25$
- $P_{Vehículo_{i,k}} = 0.1$

Donde $\Sigma_{x=1}^6 P_{x}=1, x \in \{BarriosCercanos_j, Densidad_j, Capacidad_j, Estaciones Cerca_j, Incidencias Turno_k, Vehículo_{i,k}\}$

---

Por otra parte, la aplicación con las variables del modelo, quedaría de la siguiente forma:

$maxZ = S_{1,1,1}*X_{1,1,1} + S_{2,1,1}*X_{2,1,1} + S_{3,1,1}*X_{3,1,1} + S_{4,1,1}*X_{4,1,1} + S_{5,1,1}*X_{5,1,1} + S_{1,1,2}*X_{1,1,2} + S_{2,1,2}*X_{2,1,2} + S_{3,1,2}*X_{3,1,2} + S_{4,1,2}*X_{4,1,2} + S_{5,1,2}*X_{5,1,2} + ... + S_{5,218,2}*X_{5,218,2} $

De esta forma, tendríamos 10 scorings distintos para cada estación, 5 para cada turno, siendo cada uno de ellos para un tipo de vehículo por turno.

## Importar Librerías

In [1]:
import json
import requests
import numpy as np

## Funciones

In [2]:
def load_json(lista_urls):
    """Carga el json de la url.
    - lista_urls (list): lista de url con los json a descargar.
    Devuelve una lista con los json generados.
    """
    return (json.loads(requests.get(url).text) for url in lista_urls)

def create_dic_density(barrios):
    """Crea el diccionario con densidades de todos los barrios.
    - barrios (json): json de barrios.
    Devuelve un diccionario con id de barrio como clave y densidad como valor.
    """
    dic_density = {} # diccionario de densidades.
    for b in barrios:
        dic_density[b['nta']] = round( (b['population'] / b['shape_area'])*100 , 5)
    return dic_density

def create_dic_incid(incidentes):
    """Calcula un diccionario de incidentes por barrio.
    - incidentes (json): json de incidencias.
    Devuelve un diccionario de incidentes con la siguiente estructura:
    {id_barrio_1: {
        'noche': {'num_incidentes':X, 'incidente_1': [X,X,X,X,X], 'incidente_2': [X,X,X,X,X], ...}
        'dia': {'num_incidentes':X, 'incidente_1': [X,X,X,X,X], 'incidente_2': [X,X,X,X,X], ...}
                },..}
    """
    dic_incid = {} # diccionario de incidentes.
    for i in incidentes:
        barri = i['nta'] # barrio del incidente
        if barri not in dic_incid: # si el barrio no se guardó aún
            dic_incid[barri] = {} # dic dentro de cada barrio
            # incidentes de noche y de día con dic de conteo de incidentes
            dic_incid[barri]['noche'] = {'num_incid': 0}
            dic_incid[barri]['dia'] = {'num_incid': 0}
        turno = 'noche'
        if i['is_first_shift'] == False: # si el incidente es de día
            turno = 'dia'

        dic_incid[barri][turno]['num_incid'] += 1 # aumentamos contador

        incid_pos = dic_incid[barri][turno]['num_incid']
        # cogemos valor de incidente y lo usamos para crear lista de tipos de vehículos
        # cada posición de la lista es un tipo de vehículo.
        dic_incid[barri][turno][f'incidente_{incid_pos}'] = [0,0,0,0,0]

        units = i['units'] # guardamos tipos de vehiculo del incidente
        for u in set(units): # set para sacar valores únicos
            # guardamos valores de vaca tipo de vehículo por incidente
            if u == 'engine':
              dic_incid[barri][turno][f'incidente_{incid_pos}'][0] += units.count(u)
            elif u == 'ladder':
              dic_incid[barri][turno][f'incidente_{incid_pos}'][1] += units.count(u)
            elif u == 'rescue':
              dic_incid[barri][turno][f'incidente_{incid_pos}'][2] += units.count(u)
            elif u == 'squad':
              dic_incid[barri][turno][f'incidente_{incid_pos}'][3] += units.count(u)
            elif u == 'hazardous':
              dic_incid[barri][turno][f'incidente_{incid_pos}'][4] += units.count(u)
    return dic_incid

def create_dic_estaciones(estaciones, dist_est_barrios, dic_density, dic_incid, dist_est):
    dic_ests  = {} # diccionario para guardar las estaciones cercanas a cada
    n_est = [] # nombre de las estaciones
    bc, dens, i_noc, i_dia, v_noc, v_dia, caps, ec = [],[],[],[],[],[],[],[]

    for i in estaciones:
        ### NOMBRE DE LA ESTACIÓN
        dir = i['address']
        dic_ests[dir] = {}

        n_est.append(dir)

        ### BARRIOS CERCANOS (<= 780 segs)
        dic_ests[dir]['barrios_cercanos'] = {}
        for b,d in dist_est_barrios[dir].items():
            if d <= 780: # menos de esta dist no en algunas no entran barrios.
                dic_ests[dir]['barrios_cercanos'][b] = [d]

        bc.append(len(dic_ests[dir]['barrios_cercanos']))

        ### BARRIO DE LA ESTACIÓN CON NOMBRE, DISTANCIA Y DENSIDAD
        dist, barrio = min((v[0],k) for k,v in dic_ests[dir]['barrios_cercanos'].items())
        dic_ests[dir]['barrio'] = [barrio, dist, dic_density[barrio]]

        dens.append(dic_density[barrio])

        ### INCIDENTES EN BARRIOS CERCANOS
        for b,d in dic_ests[dir]['barrios_cercanos'].items():
            if b in dic_incid.keys():
              d.append(dic_incid[b])
            else:
              d.append({})

        ### INCIDENTES MÁXIMOS DE DÍA Y DE NOCHE EN UN BARRIO
        incid_mean_noche, incid_mean_dia = [],[]
        for b,d in dic_ests[dir]['barrios_cercanos'].items():
          if d[1] != {}:
              if d[1]['noche'] != {}:
                  incid_mean_noche.append((d[1]['noche']['num_incid'], b))
              else:
                  incid_mean_noche.append((0, b))
              if d[1]['dia'] != {}:
                  incid_mean_dia.append((d[1]['dia']['num_incid'], b))
              else:
                  incid_mean_dia.append((0, b))
          else:
              incid_mean_noche.append((0, b))
              incid_mean_dia.append((0, b))

        incid_mean_noche = int(np.mean([x[0] for x in incid_mean_noche]))
        incid_mean_dia = int(np.mean([x[0] for x in incid_mean_dia]))

        dic_ests[dir]['incidentes_max'] = {'noche': incid_mean_noche,
                                          'dia': incid_mean_dia }

        i_noc.append(incid_mean_noche)
        i_dia.append(incid_mean_dia)

        ### TIPOS DE VEHÍCULOS EN BASE A INCIDENTES
        for b,d in dic_ests[dir]['barrios_cercanos'].items():
          if d[1] != {}:
              mean_vh_noche = np.mean(np.array([ v if (k != 'num_incid' and d[1]['noche'] is not None) else [0,0,0,0,0]
                                                          for k,v in d[1]['noche'].items() ]), axis=0)
              mean_vh_dia = np.mean(np.array([ v if (k != 'num_incid' and d[1]['dia'] is not None) else [0,0,0,0,0]
                                                          for k,v in d[1]['dia'].items() ]), axis=0)
          else:
              continue
          dic_ests[dir]['media_tipo_vhs'] = {
                  'noche': list(mean_vh_noche),
                  'dia': list(mean_vh_dia)
              }

        v_noc.append(list(mean_vh_noche))
        v_dia.append(list(mean_vh_dia))

        ### CAPACIDAD DE LA ESTACIÓN
        dic_ests[dir]['capacidad'] = i['capacity']

        caps.append(dic_ests[dir]['capacidad'])

        ### ESTACIONES CERCANAS A ESA ESTACIÓN (<= 700 seg)
        dic_ests[dir]['estaciones_cercanas'] = []
        for e,d in dist_est[dir].items():
            if d <= 700:
                dic_ests[dir]['estaciones_cercanas'].append((e,d))

        ec.append(len(dic_ests[dir]['estaciones_cercanas']))
    return bc, dens, i_noc, i_dia, caps, ec, v_noc, v_dia, dic_ests, n_est

### NORMALIZAR SCORES PARA VALORES ÚNICOS
def normalize_list(numbers):
    min_val = min(numbers)
    max_val = max(numbers)
    normalized = [(x - min_val) / (max_val - min_val) for x in numbers]
    return normalized

### NORMALIZAR INVERSA SCORES PARA VALORES ÚNICOS
def normalize_list_invers(numbers):
    min_val = min(numbers)
    max_val = max(numbers)
    normalized = [1-((x - min_val) / (max_val - min_val)) for x in numbers]
    return normalized

### NORMALIZAR SCORES PARA VALORES EN LISTAS
def normalize_list_of_lists(list_of_lists):
    normalized_list_of_lists = []
    for sublist in list_of_lists:
        if all(sublist) == False:
            normalized_sublist = [0.2]*5
        else:
            min_val = min(sublist)
            max_val = max(sublist)
            normalized_sublist = [1-((x - min_val) / (max_val - min_val)) for x in sublist]
        normalized_list_of_lists.append(normalized_sublist)
    return normalized_list_of_lists

def scores_norm(l_scores_1, l_scores_2):
    list_scores_norm = []
    for i,l in enumerate(l_scores_1):
        if i == 4 or i == 5:
            list_scores_norm.append(normalize_list_invers(l))
        else:
            list_scores_norm.append(normalize_list(l))
    for l in l_scores_2:
        list_scores_norm.append(normalize_list_of_lists(l))
    return list_scores_norm

### AÑADIR SCORES A CADA ESTACIÓN
def add_scores_to_station(n_est, dic_estaciones, list_scores_norm):
    dic_ests = dic_estaciones.copy()
    for e in range(len(n_est)):
        dic_ests[n_est[e]]['scorings'] = {
            'num_barrios_cerca': list_scores_norm[0][e],
            'densidad': list_scores_norm[1][e],
            'mean_incids_noche': list_scores_norm[2][e],
            'mean_incids_dia': list_scores_norm[3][e],
            'capacidad': list_scores_norm[4][e],
            'num_estaciones_cerca': list_scores_norm[5][e],
            'vhs_1_noche': list_scores_norm[6][e][0],
            'vhs_2_noche': list_scores_norm[6][e][1],
            'vhs_3_noche': list_scores_norm[6][e][2],
            'vhs_4_noche': list_scores_norm[6][e][3],
            'vhs_5_noche': list_scores_norm[6][e][4],
            'vhs_1_dia': list_scores_norm[7][e][0],
            'vhs_2_dia': list_scores_norm[7][e][1],
            'vhs_3_dia': list_scores_norm[7][e][2],
            'vhs_4_dia': list_scores_norm[7][e][3],
            'vhs_5_dia': list_scores_norm[7][e][4]
        }
    return dic_ests

def find_min_scoring(results, turno):
    return [ abs(min(v[turno])) for i in range(5) for v in results.values() ]

def find_max_scoring(results, turno):
    return [ abs(max(v[turno])) for i in range(5) for v in results.values() ]

def calcular_scoring_general(dic_ests, importancias):
    resultados = {}

    for nombre, datos in dic_ests.items():
        scorings = datos['scorings']
        scoring_noche = []
        for i in range(5):
          # Calcular el scoring para la noche
            s_noche = (
                scorings['num_barrios_cerca'] * importancias['num_barrios_cerca'] +
                scorings['densidad'] * importancias['densidad'] +
                scorings['mean_incids_noche'] * importancias['mean_incids_noche'] +
                scorings['capacidad'] * importancias['capacidad'] +
                scorings['num_estaciones_cerca'] * importancias['num_estaciones_cerca'] +
                scorings[f'vhs_{i+1}_noche'] * importancias[f'vhs_{i+1}_noche']
            )
            scoring_noche.append(s_noche)

        scoring_dia = []
        # Calcular el scoring para el día
        for i in range(5):
            s_dia = (
                scorings['num_barrios_cerca'] * importancias['num_barrios_cerca'] +
                scorings['densidad'] * importancias['densidad'] +
                scorings['mean_incids_dia'] * importancias['mean_incids_dia'] +
                scorings['capacidad'] * importancias['capacidad'] +
                scorings['num_estaciones_cerca'] * importancias['num_estaciones_cerca'] +
                scorings[f'vhs_{i+1}_dia'] * importancias[f'vhs_{i+1}_dia']
            )
            scoring_dia.append(s_dia)

        resultados[nombre] = {'scoring_noche': scoring_noche, 'scoring_dia': scoring_dia}

        # Paso 2: Encontrar los valores mínimos y máximos
        min_scorings = find_min_scoring(resultados, 'scoring_noche')
        max_scorings = find_max_scoring(resultados, 'scoring_noche')
        min_scorings_dia = find_min_scoring(resultados, 'scoring_dia')
        max_scorings_dia = find_max_scoring(resultados, 'scoring_dia')

        # Paso 3: Aplicar normalización min-max
        for i in range(5):
            scoring_noche = resultados[nombre]['scoring_noche'][i]
            scoring_dia = resultados[nombre]['scoring_dia'][i]
            resultados[nombre]['scoring_noche'][i] = (scoring_noche - min_scorings[i]) / (max_scorings[i] - min_scorings[i]) if max_scorings[i] != min_scorings[i] else 1
            resultados[nombre]['scoring_dia'][i] = (scoring_dia - min_scorings_dia[i]) / (max_scorings_dia[i] - min_scorings_dia[i]) if max_scorings_dia[i] != min_scorings_dia[i] else 1

    return resultados

def create_final_scores(scorings_generales, estaciones):
    scores = {}
    for i,e in enumerate(estaciones):
        for pos in range(5):
            scores[(pos+1,i+1,1)] = scorings_generales[e['address']]['scoring_noche'][pos]
            scores[(pos+1,i+1,2)] = scorings_generales[e['address']]['scoring_dia'][pos]
    return scores

## Generación de Scores

In [3]:
lista_urls = [
    "https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json",
    "https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json",
    "https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones.json",
    "https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/barrios.json",
    "https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json"
]

# jsons con la información necesaria para crear los scorings
incidentes, dist_est_barrios, dist_est, barrios, estaciones = load_json(lista_urls)

# diccionario de densidades de barrios precalculado
dic_density = create_dic_density(barrios)

# diccionario de incidentes precalculado
dic_incid = create_dic_incid(incidentes)

# diccionario de estaciones con toda la información relevante
result = create_dic_estaciones(estaciones, dist_est_barrios, dic_density, dic_incid, dist_est)

list_scores_1 = [ result[0], result[1], result[2], result[3], result[4], result[5] ] # VALORES ÚNICOS
list_scores_2 = [ result[6], result[7] ] # LISTA DE VALORES

# lista de scores normalizados
list_scores_norm = scores_norm(list_scores_1, list_scores_2)

# diccionario de estaciones con scores
dic_ests = add_scores_to_station(result[9], result[8], list_scores_norm)

In [4]:
# definir las importancias de cada scoring
importancias = {
    'num_barrios_cerca': 0.25,
    'densidad': 0.05,
    'mean_incids_noche': 0.25,
    'mean_incids_dia': 0.25,
    'capacidad': 0.15,
    'num_estaciones_cerca': 0.2,
    'vhs_1_noche': 0.1,
    'vhs_2_noche': 0.1,
    'vhs_3_noche': 0.1,
    'vhs_4_noche': 0.1,
    'vhs_5_noche': 0.1,
    'vhs_1_dia': 0.1,
    'vhs_2_dia': 0.1,
    'vhs_3_dia': 0.1,
    'vhs_4_dia': 0.1,
    'vhs_5_dia': 0.1
}

# calcula scores con pesos asociados
scorings_generales = calcular_scoring_general(dic_ests, importancias)

# genera los scores finales para el modelo
scores = create_final_scores(scorings_generales, estaciones)

In [5]:
def score_station(estaciones, nombre):
    for i,e in enumerate(estaciones):
        if e['address'] == nombre:
            for x in range(5):
                print(scores[(x+1,i+1,1)])
                print(scores[(x+1,i+1,2)])

In [6]:
from ortools.linear_solver import pywraplp

In [7]:
id_estaciones={}
i=1
for dic in estaciones:
    dic['Station_ID']=i
    id_estaciones[dic['address']]=i
    i+=1

In [8]:
capacities={}
for i in estaciones:
    capacities[i['Station_ID']]=i['capacity']
distancia_estaciones_barrios={}
for est,dic in dist_est_barrios.items():
    dicsort = dict(sorted(dic.items(), key=lambda item: item[1]))
    dist_est_barrios[est]=dicsort

mas_cercanos={}
for k,v in dist_est_barrios.items():
    mas_cercanos[k]=list(v.keys())[:1]
mas_cercanos
barrio_distrito={}
for dic in barrios:
    barrio_distrito[dic['nta']]=dic['boro_name']

In [9]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/located_firehouses.json")
json_response = json.loads(response.text)

coordenadas=[]
for dic in (json_response['features']):
    info={}
    info['Name']=dic['properties']['FacilityAddress']
    info['ID']=int(dic['id'])+1
    info['Lat']=dic['geometry']['coordinates'][1]
    info['Lon']=dic['geometry']['coordinates'][0]
    coordenadas.append(info)
for c in coordenadas:
    c['Barrio']=mas_cercanos[c['Name']][0]
for c in coordenadas:
    c['Distrito']=barrio_distrito[c['Barrio']]
coordenadas

[{'Name': '42 South Street',
  'ID': 1,
  'Lat': 40.703694,
  'Lon': -74.007717,
  'Barrio': 'MN25',
  'Distrito': 'Manhattan'},
 {'Name': '49 Beekman Street',
  'ID': 2,
  'Lat': 40.709971,
  'Lon': -74.005395,
  'Barrio': 'MN24',
  'Distrito': 'Manhattan'},
 {'Name': '100 Duane Street',
  'ID': 3,
  'Lat': 40.715339,
  'Lon': -74.0063,
  'Barrio': 'MN24',
  'Distrito': 'Manhattan'},
 {'Name': '14 N. Moore Street',
  'ID': 4,
  'Lat': 40.719574,
  'Lon': -74.00662,
  'Barrio': 'MN24',
  'Distrito': 'Manhattan'},
 {'Name': '75 Canal Street',
  'ID': 5,
  'Lat': 40.715408,
  'Lon': -73.992834,
  'Barrio': 'MN27',
  'Distrito': 'Manhattan'},
 {'Name': '25 Pitt Street',
  'ID': 6,
  'Lat': 40.716439,
  'Lon': -73.983927,
  'Barrio': 'MN28',
  'Distrito': 'Manhattan'},
 {'Name': '222 East 2 Street',
  'ID': 7,
  'Lat': 40.721879,
  'Lon': -73.982526,
  'Barrio': 'MN28',
  'Distrito': 'Manhattan'},
 {'Name': '340 East 14 Street',
  'ID': 8,
  'Lat': 40.731494,
  'Lon': -73.983407,
  'Barrio

In [10]:
dist_est2={}
for estacion,distancias in dist_est.items():
    dist_est2[id_estaciones[estacion]]=distancias

dic_distancias={}
for estacion,dic in dist_est2.items():
    dics2={}
    for k,v in dic.items():
        dics2[id_estaciones[k]]=v
    dic_distancias[estacion]=dics2
distancias_dic={(ext, k_int): v_int for ext, int_dict in dic_distancias.items() for k_int, v_int in int_dict.items()}
distancias_dic

{(1, 2): 154.94,
 (1, 3): 299.43,
 (1, 4): 408.12,
 (1, 5): 296.75,
 (1, 6): 392.35,
 (1, 7): 505.01,
 (1, 8): 600.54,
 (1, 9): 408.56,
 (1, 10): 494.1,
 (1, 11): 609.1,
 (1, 12): 654.99,
 (1, 13): 879.21,
 (1, 14): 841.05,
 (1, 15): 759.44,
 (1, 16): 841.22,
 (1, 17): 771.14,
 (1, 18): 731.46,
 (1, 19): 773.22,
 (1, 20): 899.85,
 (1, 21): 967.4,
 (1, 22): 1059.49,
 (1, 23): 1151.78,
 (1, 24): 1236.84,
 (1, 25): 763.81,
 (1, 26): 975.39,
 (1, 27): 1001.09,
 (1, 28): 1248.45,
 (1, 29): 1091.13,
 (1, 30): 1204.23,
 (1, 31): 1289.21,
 (1, 32): 1348.47,
 (1, 33): 1393.09,
 (1, 34): 1362.0,
 (1, 35): 1441.98,
 (1, 36): 1606.89,
 (1, 37): 1661.13,
 (1, 38): 366.57,
 (1, 39): 948.64,
 (1, 40): 866.7,
 (1, 41): 1479.4,
 (1, 42): 1496.46,
 (1, 43): 1536.17,
 (1, 44): 175.02,
 (1, 45): 831.43,
 (1, 46): 517.35,
 (1, 47): 1598.08,
 (1, 48): 699.88,
 (1, 49): 1377.38,
 (1, 50): 1507.81,
 (1, 51): 1551.37,
 (1, 52): 1656.6,
 (1, 53): 1534.05,
 (1, 54): 1331.22,
 (1, 55): 1607.31,
 (1, 56): 1773.81,

# CREACIÓN DEL SOLVER

In [11]:
solver = pywraplp.Solver.CreateSolver("GLOP")
types_vehicles=5
number_stations=len(estaciones)
turnos=2
ubicaciones = {}
cambios={}

for i in range(types_vehicles):
    for j in range(number_stations):
        for k in range(turnos):
            v = solver.BoolVar(f'Vehicle of type {i+1} assigned at station {j+1} at shift {k+1}')
            ubicaciones[(i+1,j+1,k+1)] = v
for i in range(types_vehicles):
    for j in range(number_stations):
        for k in range(number_stations):
            if j!=k:
                v = solver.BoolVar(f'Vehicle of type {i+1} moves from station {j+1} to station {k+1}')
                cambios[(i+1,j+1,k+1)] = v

### Creación de la función objetivo

In [12]:
objective = solver.Objective()
objective.SetMaximization()
for i in range(types_vehicles):
    for j in range(number_stations):
        for k in range(turnos):
            v = ubicaciones[(i+1,j+1,k+1)]
            score = scores[(i+1,j+1,k+1)]
            objective.SetCoefficient(v, score)
for i in range(types_vehicles):
    for j in range(number_stations):
        for k in range(number_stations):
            if j!=k:
                c = cambios[(i+1,j+1,k+1)]
                score = 0
                objective.SetCoefficient(c, score)

### El número total de vehiculos asignados en una estación debe ser mínimo 1 y máximo la capacidad de la estación

In [13]:
capacity_constraints=[]
for i in range(number_stations):
    for k in range(turnos):
        cap=capacities[i+1]
        c = solver.Constraint(1, cap)
        for j in range(types_vehicles):
            u=ubicaciones[(j+1,i+1,k+1)]
            c.SetCoefficient(u, 1)
        capacity_constraints.append(c)

### El número de vehiculos asignados de cada tipo debe ser exactamente el número de vehiculos disponibles de cada tipo

In [14]:
number_vehicles={1:197,2:143,3:5,4:8,5:1}
vehicle_shift_constraints=[]
for i in range(types_vehicles):
    for k in range(turnos):
        c = solver.Constraint(number_vehicles[i+1], number_vehicles[i+1])
        for j in range(number_stations):
            u=ubicaciones[(i+1,j+1,k+1)]
            c.SetCoefficient(u, 1)
        vehicle_shift_constraints.append(c)

### No puede salir un vehiculo de un tipo si no está en la estación en el primer turno, si lo había, solo puede moverse uno de cada estación

In [15]:
exit_constraints=[]
for i in range(types_vehicles):
    for j in range(number_stations):
        c = solver.Constraint(-solver.infinity(),0)
        ubi=ubicaciones[(i+1,j+1,1)]
        c.SetCoefficient(ubi,-1)
        for k in range(number_stations):
            if k!=j:
                cmb=cambios[(i+1,j+1,k+1)]
                c.SetCoefficient(cmb, 1)
        exit_constraints.append(c)

### No puede llegar un vehiculo de un tipo si ya lo había en el turno 1, si no lo había puede llegar máximo 1 de ese tipo

In [16]:
arrive_constraints=[]
for i in range(types_vehicles):
    for k in range(number_stations):
        c=solver.Constraint(-solver.infinity(),1)
        ubi=ubicaciones[(i+1,k+1,1)]
        c.SetCoefficient(ubi,1)
        for j in range(number_stations):
            if j!=k:
                cam=cambios[(i+1,j+1,k+1)]
                c.SetCoefficient(cam,1)
        arrive_constraints.append(c)

### Turno2 = Turno1 - salidas + entradas

In [17]:
constraints=[]
for i in range(types_vehicles):
    for j in range(number_stations):
        c=solver.Constraint(0,0)
        turno2=ubicaciones[(i+1,j+1,2)]
        turno1=ubicaciones[(i+1,j+1,1)]
        c.SetCoefficient(turno2,1)
        c.SetCoefficient(turno1,-1)
        for k in range(number_stations):
            if k!=j:
                salidas=cambios[(i+1,j+1,k+1)]
                c.SetCoefficient(salidas,1)
                entradas=cambios[(i+1,k+1,j+1)]
                c.SetCoefficient(entradas,-1)
        constraints.append(c)

### NO PUEDE HABER MAS DE DOS VEHICULOS TIPO 3 EN EL MISMO DISTRITO

In [18]:

manhattan=[]
for estacion in coordenadas:
    if estacion['Distrito']=='Manhattan':
        manhattan.append(estacion['ID'])
for k in range(turnos):
    c=solver.Constraint(0,2)
    for j in range(number_stations):
        if j+1 in manhattan:
            v=ubicaciones[(3,j+1,k+1)]
            c.SetCoefficient(v,1)
bronx=[]
for estacion in coordenadas:
    if estacion['Distrito']=='Bronx':
        bronx.append(estacion['ID'])
for k in range(turnos):
    c=solver.Constraint(0,2)
    for j in range(number_stations):
        if j+1 in bronx:
            v=ubicaciones[(3,j+1,k+1)]
            c.SetCoefficient(v,1)
Brooklyn=[]
for estacion in coordenadas:
    if estacion['Distrito']=='Brooklyn':
        Brooklyn.append(estacion['ID'])
for k in range(turnos):
    c=solver.Constraint(0,2)
    for j in range(number_stations):
        if j+1 in Brooklyn:
            v=ubicaciones[(3,j+1,k+1)]
            c.SetCoefficient(v,1)
Queens=[]
for estacion in coordenadas:
    if estacion['Distrito']=='Queens':
        Queens.append(estacion['ID'])
for k in range(turnos):
    c=solver.Constraint(0,2)
    for j in range(number_stations):
        if j+1 in Queens:
            v=ubicaciones[(3,j+1,k+1)]
            c.SetCoefficient(v,1)
Staten_Island=[]
for estacion in coordenadas:
    if estacion['Distrito']=='Staten Island':
        Staten_Island.append(estacion['ID'])
for k in range(turnos):
    c=solver.Constraint(0,2)
    for j in range(number_stations):
        if j+1 in Staten_Island:
            v=ubicaciones[(3,j+1,k+1)]
            c.SetCoefficient(v,1)


### Como máximo puede haber 3 coches tipo 4 en cada distrito y mímino 1

In [19]:

for k in range(turnos):
    c=solver.Constraint(1,3)
    for j in range(number_stations):
        if j+1 in manhattan:
            v=ubicaciones[(4,j+1,k+1)]
            c.SetCoefficient(v,1)
for k in range(turnos):
    c=solver.Constraint(1,3)
    for j in range(number_stations):
        if j+1 in bronx:
            v=ubicaciones[(4,j+1,k+1)]
            c.SetCoefficient(v,1)

for k in range(turnos):
    c=solver.Constraint(1,3)
    for j in range(number_stations):
        if j+1 in Brooklyn:
            v=ubicaciones[(4,j+1,k+1)]
            c.SetCoefficient(v,1)

for k in range(turnos):
    c=solver.Constraint(1,3)
    for j in range(number_stations):
        if j+1 in Queens:
            v=ubicaciones[(4,j+1,k+1)]
            c.SetCoefficient(v,1)
for k in range(turnos):
    c=solver.Constraint(1,3)
    for j in range(number_stations):
        if j+1 in Staten_Island:
            v=ubicaciones[(4,j+1,k+1)]
            c.SetCoefficient(v,1)

### Los moviminientos no pueden ser mas lejanos a una distancia de 780 segundos

In [20]:
for i in range(types_vehicles):
    for j in range(number_stations):
        for k in range(number_stations):
            if j!=k:
                if distancias_dic[(j+1,k+1)]>600:
                    c=solver.Constraint(0,0)
                    v=cambios[(i+1,j+1,k+1)]
                    c.SetCoefficient(v,1)

In [21]:
dic_distancias

{1: {2: 154.94,
  3: 299.43,
  4: 408.12,
  5: 296.75,
  6: 392.35,
  7: 505.01,
  8: 600.54,
  9: 408.56,
  10: 494.1,
  11: 609.1,
  12: 654.99,
  13: 879.21,
  14: 841.05,
  15: 759.44,
  16: 841.22,
  17: 771.14,
  18: 731.46,
  19: 773.22,
  20: 899.85,
  21: 967.4,
  22: 1059.49,
  23: 1151.78,
  24: 1236.84,
  25: 763.81,
  26: 975.39,
  27: 1001.09,
  28: 1248.45,
  29: 1091.13,
  30: 1204.23,
  31: 1289.21,
  32: 1348.47,
  33: 1393.09,
  34: 1362.0,
  35: 1441.98,
  36: 1606.89,
  37: 1661.13,
  38: 366.57,
  39: 948.64,
  40: 866.7,
  41: 1479.4,
  42: 1496.46,
  43: 1536.17,
  44: 175.02,
  45: 831.43,
  46: 517.35,
  47: 1598.08,
  48: 699.88,
  49: 1377.38,
  50: 1507.81,
  51: 1551.37,
  52: 1656.6,
  53: 1534.05,
  54: 1331.22,
  55: 1607.31,
  56: 1773.81,
  57: 1589.66,
  58: 1763.09,
  59: 1792.46,
  60: 1650.75,
  61: 1876.4,
  62: 1790.05,
  63: 1944.2,
  64: 1698.05,
  65: 1941.72,
  66: 2001.39,
  67: 2174.5,
  68: 2164.71,
  69: 2330.1,
  70: 2159.23,
  71: 2274

In [22]:
stations_1_1=[]
stations_1_2=[]
stations_2_1=[]
stations_2_2=[]
stations_3_1=[]
stations_3_2=[]
stations_4_1=[]
stations_4_2=[]
stations_5_1=[]
stations_5_2=[]

result = solver.Solve()
if result == solver.OPTIMAL :
    print("In the specified time limit the solver has found a feasible solution")
    for i in range(types_vehicles):
        for j in range(number_stations):
            for k in range(turnos):
                v = ubicaciones[(i+1,j+1,k+1)]
                
                if v.SolutionValue()>0.1:
                    if i+1==1 and k+1==1:
                        stations_1_1.append(j+1)
                    if i+1==1 and k+1==2:
                        stations_1_2.append(j+1)
                    if i+1==2 and k+1==1:
                        stations_2_1.append(j+1)
                    if i+1==2 and k+1==2:
                        stations_2_2.append(j+1)
                    if i+1==3 and k+1==1:
                        stations_3_1.append(j+1)
                    if i+1==3 and k+1==2:
                        stations_3_2.append(j+1)
                    if i+1==4 and k+1==1:
                        stations_4_1.append(j+1)
                    if i+1==4 and k+1==2:
                        stations_4_2.append(j+1)
                    if i+1==5 and k+1==1:
                        stations_5_1.append(j+1)
                    if i+1==5 and k+1==2:
                        stations_5_2.append(j+1)
                        
                    print(v, (v.solution_value()))
    print('----------------------')
    for i in range(types_vehicles):
        for j in range(number_stations):
            for k in range(number_stations):
                if j!=k:
                    c = cambios[(i+1,j+1,k+1)]
                    if c.SolutionValue()>0.1:
                        print(c, (c.solution_value()))
                        
crd={(1,1):stations_1_1,(1,2):stations_1_2,(2,1):stations_2_1,(2,2):stations_2_2,(3,1):stations_3_1,(3,2):stations_3_2,(4,1):stations_4_1,(4,2):stations_4_2,(5,1):stations_5_1,(5,2):stations_5_2}

In the specified time limit the solver has found a feasible solution
Vehicle of type 1 assigned at station 3 at shift 2 1.0
Vehicle of type 1 assigned at station 5 at shift 2 1.0
Vehicle of type 1 assigned at station 6 at shift 2 1.0
Vehicle of type 1 assigned at station 7 at shift 1 1.0
Vehicle of type 1 assigned at station 7 at shift 2 1.0
Vehicle of type 1 assigned at station 8 at shift 2 1.0
Vehicle of type 1 assigned at station 9 at shift 1 1.0
Vehicle of type 1 assigned at station 9 at shift 2 1.0
Vehicle of type 1 assigned at station 10 at shift 1 1.0
Vehicle of type 1 assigned at station 10 at shift 2 1.0
Vehicle of type 1 assigned at station 11 at shift 1 1.0
Vehicle of type 1 assigned at station 11 at shift 2 1.0
Vehicle of type 1 assigned at station 12 at shift 1 1.0
Vehicle of type 1 assigned at station 12 at shift 2 1.0
Vehicle of type 1 assigned at station 13 at shift 1 1.0
Vehicle of type 1 assigned at station 13 at shift 2 1.0
Vehicle of type 1 assigned at station 14 at

Vehicle of type 4 moves from station 62 to station 80 1.0
Vehicle of type 4 moves from station 183 to station 182 1.0
Vehicle of type 4 moves from station 212 to station 204 1.0


In [23]:
import folium
def mostrarmapa(v,t):# x es el tipo de vehiculo/ t es el turno
    mapa = folium.Map(location=[40.70, -73.94], zoom_start=10, width="50%", tiles="CartoDB positron")
    for est in crd[(v,t)]:
        lat=0
        lon=0
        for estacion in coordenadas:
            if estacion['ID']==est:
                lat+=estacion['Lat']
                lon+=estacion['Lon']
            folium.Circle(location=(lat,lon), radius=250, color='blue', fill=True, fill_color='red',fill_opacity=1).add_to(mapa)
    return mapa

C:\Users\marcs\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [36]:
mostrarmapa(5,1)